In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import scipy
from scipy.signal import butter, filtfilt, iirnotch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
import scipy.io

from tqdm import tqdm
from sklearn.metrics import f1_score, recall_score, accuracy_score, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# Importa librerie
from sklearn.model_selection import StratifiedKFold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler


# Monta Drive e imposta Device
drive.mount("/content/drive")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Impostazioni globali per la riproducibilità
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.benchmark = True
else:
    torch.manual_seed(42)
np.random.seed(42)
torch.manual_seed(42)

# ==============================================================================
# 1. PREPARAZIONE DATI E FUNZIONI DI FILTRAGGIO
# ==============================================================================

# --- Funzioni di filtraggio ---
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def apply_bandpass_filter(data, lowcut=1, highcut=40, fs=500, order=2):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    return filtfilt(b, a, data)

def notch_filter(data, freq=50, fs=500, quality_factor=30):
    b, a = iirnotch(freq / (fs / 2), quality_factor)
    return filtfilt(b, a, data)

def extract_patient_id(filename):
    return int(filename.split(".")[0])

def segment_ecg(signal, segment_length=2500):
    segments = np.zeros((signal.shape[0], segment_length, signal.shape[2]))
    for i in range(signal.shape[0]):
      start = 0 # Inizia dall'inizio
      end = start + segment_length
      segments[i, :, :] = signal[i, start:end, :]
    return segments

# --- Carica e Pre-processa Dati ---
ECG_folder = "/content/drive/MyDrive/ECG_Deep_Learning/1_batch_extracted"
ECG_folder_2batch = "/content/drive/MyDrive/ECG_Deep_Learning/2_batch_extracted"
tabular_data = pd.read_excel("/content/drive/MyDrive/ECG_Deep_Learning/VALETUDO_database_1st_batch_en_all_info.xlsx")
tabular_data_2batch = pd.read_excel("/content/drive/MyDrive/ECG_Deep_Learning/VALETUDO_database_2nd_batch_en_all_info.xlsx")

ECGs_1 = [f for f in os.listdir(ECG_folder) if f.endswith(".mat")]
ECGs_2 = [f for f in os.listdir(ECG_folder_2batch) if f.endswith(".mat")]
ECGs_1.sort(key=extract_patient_id)
ECGs_2.sort(key=extract_patient_id)

signals_1 = np.empty((len(ECGs_1), 5000, 12))
for index, ecg_path in enumerate(ECGs_1):
    filepath = os.path.join(ECG_folder, ecg_path)
    matdata = scipy.io.loadmat(filepath)
    ecg = matdata['val']
    for i in range(12):
        ecg[:, i] = ecg[:, i] - np.mean(ecg[:, i])
        ecg[:, i] = apply_bandpass_filter(ecg[:, i])
        ecg[:, i] = notch_filter(ecg[:, i])
    signals_1[index, :, :] = ecg

signals_2 = np.empty((len(ECGs_2), 5000, 12))
for index, ecg_path in enumerate(ECGs_2):
    filepath = os.path.join(ECG_folder_2batch, ecg_path)
    matdata = scipy.io.loadmat(filepath)
    ecg = matdata['val']
    for i in range(12):
        ecg[:, i] = ecg[:, i] - np.mean(ecg[:, i])
        ecg[:, i] = apply_bandpass_filter(ecg[:, i])
        ecg[:, i] = notch_filter(ecg[:, i])
    signals_2[index, :, :] = ecg

# Concatenazione
signals = np.concatenate([signals_1, signals_2], axis=0)
tabular_data = pd.concat([
    tabular_data.sort_values(by="ECG_patient_id").reset_index(drop=True),
    tabular_data_2batch.sort_values(by="ECG_patient_id").reset_index(drop=True)
], ignore_index=True)

# Pulizia dati tabulari (rimozione colonne con NaN, per semplicità)
missing_cols = tabular_data.columns[tabular_data.isnull().any()].tolist()
tabular_data = tabular_data.drop(columns=missing_cols)
print(f"Dati tabulari puliti shape: {tabular_data.shape}")

# Segmentazione ECG (2500 campioni)
ecg_segments = segment_ecg(signals, segment_length=2500)
print(f"ECG Segmenti shape: {ecg_segments.shape}")

# ==============================================================================
# 2. DEFINIZIONE DEL DATASET E DELLE ARCHITETTURE (Simple, Deep, Wide)
# ==============================================================================

class ECGDataset(Dataset):
    def __init__(self, signals, labels):
        # Permuta i segnali in (N, Channels, Length) per Conv1d
        if signals.shape[1] > signals.shape[2]:
            self.signals = torch.tensor(signals, dtype=torch.float32).permute(0, 2, 1)
        else:
            self.signals = torch.tensor(signals, dtype=torch.float32)

        self.labels = torch.tensor(labels.values, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.signals[idx], self.labels[idx]

# --- 2.1. Simple1DCNN (Architettura Originale/Riferimento) ---
class Simple1DCNN(nn.Module):
    def __init__(self, num_leads=12):
        super(Simple1DCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_leads, out_channels=32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(kernel_size=2)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(128, 64)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool1(F.gelu(self.bn1(self.conv1(x))))
        x = self.pool2(F.gelu(self.bn2(self.conv2(x))))
        x = self.pool3(F.gelu(self.bn3(self.conv3(x))))
        x = self.global_pool(x).squeeze(-1)
        x = F.gelu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

# --- 2.2. Deep1DCNN (Più Profonda) ---
class Deep1DCNN(nn.Module):
    def __init__(self, num_leads=12):
        super(Deep1DCNN, self).__init__()
        # Blocco 1-3 (come Simple)
        self.conv1 = nn.Conv1d(in_channels=num_leads, out_channels=32, kernel_size=5, padding=2); self.bn1 = nn.BatchNorm1d(32); self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, padding=2); self.bn2 = nn.BatchNorm1d(64); self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2); self.bn3 = nn.BatchNorm1d(128); self.pool3 = nn.MaxPool1d(kernel_size=2)

        # Blocco 4 (NUOVO)
        self.conv4 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, padding=2)
        self.bn4 = nn.BatchNorm1d(256)
        self.pool4 = nn.MaxPool1d(kernel_size=2)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(256, 64) # Input 256
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool1(F.gelu(self.bn1(self.conv1(x))))
        x = self.pool2(F.gelu(self.bn2(self.conv2(x))))
        x = self.pool3(F.gelu(self.bn3(self.conv3(x))))
        x = self.pool4(F.gelu(self.bn4(self.conv4(x)))) # Nuovo

        x = self.global_pool(x).squeeze(-1)
        x = F.gelu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

# --- 2.3. Wide1DCNN (Più Larga) ---
class Wide1DCNN(nn.Module):
    def __init__(self, num_leads=12):
        super(Wide1DCNN, self).__init__()
        # Blocco 1: 12 -> 64 (Era 32)
        self.conv1 = nn.Conv1d(in_channels=num_leads, out_channels=64, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm1d(64)
        self.pool1 = nn.MaxPool1d(kernel_size=2)

        # Blocco 2: 64 -> 128 (Era 64)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(kernel_size=2)

        # Blocco 3: 128 -> 256 (Era 128)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=5, padding=2)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(kernel_size=2)

        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(256, 64) # Input 256
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool1(F.gelu(self.bn1(self.conv1(x))))
        x = self.pool2(F.gelu(self.bn2(self.conv2(x))))
        x = self.pool3(F.gelu(self.bn3(self.conv3(x))))

        x = self.global_pool(x).squeeze(-1)
        x = F.gelu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

# ==============================================================================
# 3. FUNZIONE DI TRAINING E VALUTAZIONE
# ==============================================================================

def train_and_evaluate_model(ModelClass, ecg_data, tabular_data, learning_rate, num_epocs=50, batch_size=32, threshold=0.6, patience=5):
    """
    Esegue la 10-Fold Cross-Validation per un dato modello e learning rate.
    """

    # Inizializzazione della K-Fold
    strat_kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    # Liste per memorizzare le metriche di TUTTI i fold
    metrics_all_folds = {
        'f1': [], 'sensitivity': [], 'specificity': [], 'accuracy': [], 'auc': [], 'fpr': [], 'tpr': [], 'test_loss_history': [], 'train_loss_history': [],
        'train_f1': [], 'train_sensitivity': [], 'train_specificity': [], 'train_accuracy': [], 'train_auc': []
    }

    # Inizializzazione Imputer e Scaler (solo una volta)
    imputer = IterativeImputer(random_state=42)
    scaler = StandardScaler()

    for fold, (train_index, test_index) in enumerate(strat_kf.split(ecg_data, tabular_data['sport_ability'])):
        print(f"\n--- FOLD {fold+1}/10 ---")

        # 1. Suddivisione dei Dati
        ecg_train, ecg_test = ecg_data[train_index, :, :], ecg_data[test_index, :, :]
        X_train, X_test = tabular_data.iloc[train_index,:], tabular_data.iloc[test_index,:]

        Y_train = X_train['sport_ability']
        Y_test = X_test['sport_ability']

        # Prepara colonne tabulari da processare (NOTA: NON USATE NEL MODELLO, MA PROCESSATE PER COERENZA)
        cols_to_drop = ['sport_ability', 'ECG_patient_id']
        X_train_proc = X_train.drop(columns=[col for col in cols_to_drop if col in X_train.columns])
        X_test_proc = X_test.drop(columns=[col for col in cols_to_drop if col in X_test.columns])

        # Dinamicamente determina le colonne numeriche e categoriche presenti nel dataframe
        all_available_cols_proc = X_train_proc.columns.tolist()

        potential_numeric_cols = ['age_at_exam', 'height', 'weight', 'trainning_load']
        potential_categorical_cols = ['sex', 'sport_classification'] # Add other categorical columns if any

        numeric_cols = [col for col in potential_numeric_cols if col in all_available_cols_proc]
        categorical_cols = [col for col in potential_categorical_cols if col in all_available_cols_proc]

        # 2. Imputazione e Normalizzazione dei Dati Tabulari (Training)
        X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_proc), columns=X_train_proc.columns)
        if numeric_cols:
            X_train_imputed[numeric_cols] = scaler.fit_transform(X_train_imputed[numeric_cols])
        # Imputazione e Normalizzazione dei Dati Tabulari (Test)
        X_test_imputed = pd.DataFrame(imputer.transform(X_test_proc), columns=X_test_proc.columns)
        if numeric_cols:
            X_test_imputed[numeric_cols] = scaler.transform(X_test_imputed[numeric_cols])

        # 3. Creazione Dataset e DataLoader
        train_dataset = ECGDataset(ecg_train, Y_train)
        test_dataset = ECGDataset(ecg_test, Y_test)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        # 4. Inizializzazione Modello e Ottimizzatore
        model = ModelClass(num_leads=12).to(device)
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Variabili Early Stopping e Loss History
        best_test_loss = float('inf')
        trigger_times = 0
        current_train_loss_hist = []
        current_test_loss_hist = []

        # Variabili per trovare la miglior epoca del fold
        best_epoch_metrics = {'f1': 0.0, 'epoch': 0, 'test_loss': 0.0}

        # Inizializzazione metriche per epoca
        epoch_metrics = {k: [] for k in metrics_all_folds.keys() if 'history' not in k and 'train_' not in k}
        epoch_train_metrics = {k: [] for k in metrics_all_folds.keys() if 'train_' in k}

        for epoch in range(num_epocs):

            # --- TRAINING PHASE ---
            model.train()
            train_loss = 0.0
            all_labels_train, all_preds_train, all_outputs_train = [], [], []

            for signals_ecg, labels in train_loader:
                signals_ecg, labels = signals_ecg.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(signals_ecg).squeeze() # Output: [Batch Size]

                loss = criterion(outputs, labels.squeeze())
                loss.backward()
                optimizer.step()
                train_loss += loss.item()

                predicted = (outputs > threshold).int()
                all_labels_train.extend(labels.int().cpu().numpy())
                all_preds_train.extend(predicted.cpu().numpy())

            train_loss /= len(train_loader)
            current_train_loss_hist.append(train_loss)

            # Calcolo metriche di TRAIN
            train_f1 = f1_score(all_labels_train, all_preds_train)
            tn, fp, fn, tp = confusion_matrix(all_labels_train, all_preds_train).ravel()
            train_sensitivity = tp / (tp + fn + 1e-8)
            train_specificity = tn / (tn + fp + 1e-8)
            train_accuracy = accuracy_score(all_labels_train, all_preds_train) * 100
            train_auc = roc_auc_score(all_labels_train, all_preds_train)

            # --- VALIDATION/TEST PHASE ---
            model.eval()
            test_loss = 0.0
            all_labels_test, all_preds_test, all_outputs_test = [], [], []

            with torch.no_grad():
                for signals_ecg, labels in test_loader:
                    signals_ecg, labels = signals_ecg.to(device), labels.to(device)
                    outputs = model(signals_ecg).squeeze()

                    loss = criterion(outputs, labels.squeeze())
                    test_loss += loss.item()

                    predicted = (outputs > threshold).int()
                    all_labels_test.extend(labels.int().cpu().numpy())
                    all_preds_test.extend(predicted.cpu().numpy())
                    all_outputs_test.extend(outputs.cpu().numpy())

            test_loss /= len(test_loader)
            current_test_loss_hist.append(test_loss)

            # Calcolo metriche di TEST
            test_f1 = f1_score(all_labels_test, all_preds_test)
            tn, fp, fn, tp = confusion_matrix(all_labels_test, all_preds_test).ravel()
            test_sensitivity = tp / (tp + fn + 1e-8)
            test_specificity = tn / (tn + fp + 1e-8)
            test_accuracy = accuracy_score(all_labels_test, all_preds_test) * 100
            test_auc = roc_auc_score(all_labels_test, all_outputs_test) # AUC usa gli output non binarizzati

            # Curva ROC per il plot finale
            fpr, tpr, _ = roc_curve(all_labels_test, all_outputs_test)

            # Memorizza metriche dell'epoca
            epoch_metrics['f1'].append(test_f1)
            epoch_metrics['sensitivity'].append(test_sensitivity)
            epoch_metrics['specificity'].append(test_specificity)
            epoch_metrics['accuracy'].append(test_accuracy)
            epoch_metrics['auc'].append(test_auc)
            epoch_metrics['fpr'].append(fpr)
            epoch_metrics['tpr'].append(tpr)

            # Memorizza metriche di TRAIN per l'epoca
            epoch_train_metrics['train_f1'].append(train_f1)
            epoch_train_metrics['train_sensitivity'].append(train_sensitivity)
            epoch_train_metrics['train_specificity'].append(train_specificity)
            epoch_train_metrics['train_accuracy'].append(train_accuracy)
            epoch_train_metrics['train_auc'].append(train_auc)

            # Aggiorna la miglior epoca in base al F1-Score
            if test_f1 > best_epoch_metrics['f1']:
                best_epoch_metrics['f1'] = test_f1
                best_epoch_metrics['epoch'] = epoch
                best_epoch_metrics['test_loss'] = test_loss
                # Salviamo i pesi del modello con il miglior F1
                torch.save(model.state_dict(), f'best_model_fold_{fold}.pth')

            # --- Early Stopping basato sulla Test Loss ---
            if test_loss < best_test_loss:
                 best_test_loss = test_loss
                 trigger_times = 0
            else:
                 trigger_times += 1
                 if trigger_times >= patience:
                     print(f"Early stopping attivato all'epoca {epoch+1}!")
                     # Ricarica i pesi migliori (basati sulla Loss, non sull'F1)
                     # Per il confronto finale useremo l'epoca con F1 massimo.
                     break

        # 5. Fine Fold: Aggrega risultati della miglior epoca (basata su F1)
        best_epoch_index = best_epoch_metrics['epoch']

        print(f"Fold {fold+1} terminato. Miglior F1 all'epoca {best_epoch_index+1}: {best_epoch_metrics['f1']:.4f}")

        # Aggiungi le metriche del FOLD (usando l'epoca con il miglior F1)
        metrics_all_folds['f1'].append(epoch_metrics['f1'][best_epoch_index])
        metrics_all_folds['sensitivity'].append(epoch_metrics['sensitivity'][best_epoch_index])
        metrics_all_folds['specificity'].append(epoch_metrics['specificity'][best_epoch_index])
        metrics_all_folds['accuracy'].append(epoch_metrics['accuracy'][best_epoch_index])
        metrics_all_folds['auc'].append(epoch_metrics['auc'][best_epoch_index])
        metrics_all_folds['fpr'].append(epoch_metrics['fpr'][best_epoch_index])
        metrics_all_folds['tpr'].append(epoch_metrics['tpr'][best_epoch_index])

        # Aggiungi le metriche di TRAIN per il confronto
        metrics_all_folds['train_f1'].append(epoch_train_metrics['train_f1'][best_epoch_index])
        metrics_all_folds['train_sensitivity'].append(epoch_train_metrics['train_sensitivity'][best_epoch_index])
        metrics_all_folds['train_specificity'].append(epoch_train_metrics['train_specificity'][best_epoch_index])
        metrics_all_folds['train_accuracy'].append(epoch_train_metrics['train_accuracy'][best_epoch_index])
        metrics_all_folds['train_auc'].append(epoch_train_metrics['train_auc'][best_epoch_index])

        # Aggiungi le loss history COMPLETE per i grafici
        metrics_all_folds['train_loss_history'].append(current_train_loss_hist)
        metrics_all_folds['test_loss_history'].append(current_test_loss_hist)

    return metrics_all_folds

# ==============================================================================
# 4. ESECUZIONE DEI TRE MODELLI
# ==============================================================================

# Dizionario per memorizzare i risultati finali di tutti i modelli
final_results = {}
configurations = [
    {'name': 'Simple1DCNN (LR 1e-3)', 'model': Simple1DCNN, 'lr': 1e-3, 'threshold': 0.6},
    {'name': 'Deep1DCNN (LR 1e-3)', 'model': Deep1DCNN, 'lr': 1e-3, 'threshold': 0.6},
    {'name': 'Wide1DCNN (LR 5e-4)', 'model': Wide1DCNN, 'lr': 5e-4, 'threshold': 0.6},
]

for config in configurations:
    print(f"\n\n=======================================================")
    print(f"INIZIO ESECUZIONE: {config['name']}")
    print(f"=======================================================")

    # Esegui la 10-Fold CV
    metrics = train_and_evaluate_model(
        ModelClass=config['model'],
        ecg_data=ecg_segments,
        tabular_data=tabular_data,
        learning_rate=config['lr'],
        threshold=config['threshold']
    )

    # Prepara il DataFrame per il confronto
    df_metrics = pd.DataFrame({
        'Accuracy': np.array(metrics['accuracy']) / 100,
        'F1 Score': metrics['f1'],
        'Sensitivity (Recall)': metrics['sensitivity'],
        'Specificity': metrics['specificity'],
        'AUC': metrics['auc'],
    })

    # Memorizza i risultati
    final_results[config['name']] = {
        'df_results': df_metrics,
        'metrics_raw': metrics,
        'summary': df_metrics.describe().loc[['mean', 'std']]
    }

    print(f"\nRISULTATI MEDI {config['name']}:")
    print(final_results[config['name']]['summary'].round(4))

# ==============================================================================
# 5. CONFRONTO E VISUALIZZAZIONE FINALE
# ==============================================================================

def plot_comparison(results_dict):
    """Genera boxplot per il confronto delle metriche tra tutti i modelli."""

    # Concatena tutti i DataFrame dei risultati
    df_list = []
    for name, data in results_dict.items():
        df = data['df_results'].copy()
        df['Configuration'] = name
        df_list.append(df)

    df_all = pd.concat(df_list, ignore_index=True)
    df_melt = df_all.melt(id_vars='Configuration', var_name='Metric', value_name='Score')

    # Visualizzazione Boxplot
    plt.figure(figsize=(16, 7))
    sns.boxplot(x='Metric', y='Score', hue='Configuration', data=df_melt, palette="Set2")

    plt.title('Confronto delle Performance (10-Fold CV) tra Architetture CNN')
    plt.ylabel('Valore Metrica')
    plt.xlabel('Metrica di Valutazione')
    plt.legend(title='Modello', loc='lower right')
    plt.grid(True, axis='y', alpha=0.3)
    plt.ylim(0, 1.05)
    plt.tight_layout()
    plt.show()



    # Visualizzazione Curve ROC Medie
    plt.figure(figsize=(10, 8))
    mean_fpr = np.linspace(0, 1, 100)

    for name, data in results_dict.items():
        metrics = data['metrics_raw']
        tprs = []
        aucs = []

        # Calcola la media delle curve ROC
        for i in range(len(metrics['fpr'])):
            interp_tpr = np.interp(mean_fpr, metrics['fpr'][i], metrics['tpr'][i])
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            aucs.append(metrics['auc'][i])

        mean_tpr = np.mean(tprs, axis=0)
        mean_auc = np.mean(aucs)
        std_auc = np.std(aucs)

        plt.plot(
            mean_fpr,
            mean_tpr,
            label=f'{name} (Mean AUC={mean_auc:.3f} \u00b1 {std_auc:.3f})',
            linewidth=2
        )

    plt.plot([0, 1], [0, 1], color='black', linestyle='--', label='Chance')
    plt.xlabel('False Positive Rate (FPR)')
    plt.ylabel('True Positive Rate (TPR)')
    plt.title('Curva ROC Media a Confronto (10-Fold CV)')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()



# Esecuzione del Confronto Grafico
plot_comparison(final_results)

# Stampa il riepilogo finale
print("\n\n#######################################################")
print("##### RIEPILOGO FINALE DELLE PERFORMANCE MEDIE #####")
print("#######################################################")

summary_list = []
for name, data in final_results.items():
    summary_df = data['summary'].copy().T
    summary_df['Model'] = name
    summary_list.append(summary_df)

df_final_summary = pd.concat(summary_list).set_index(['Model', summary_list[0].index.name])
df_final_summary.columns = ['Mean', 'Std']
print(df_final_summary.round(4).to_markdown())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cpu
Dati tabulari puliti shape: (526, 15)
ECG Segmenti shape: (526, 2500, 12)


INIZIO ESECUZIONE: Simple1DCNN (LR 1e-3)

--- FOLD 1/10 ---
Early stopping attivato all'epoca 11!
Fold 1 terminato. Miglior F1 all'epoca 2: 0.8090

--- FOLD 2/10 ---
Early stopping attivato all'epoca 18!
Fold 2 terminato. Miglior F1 all'epoca 12: 0.8500

--- FOLD 3/10 ---
Early stopping attivato all'epoca 12!
Fold 3 terminato. Miglior F1 all'epoca 3: 0.8182

--- FOLD 4/10 ---
Early stopping attivato all'epoca 19!
Fold 4 terminato. Miglior F1 all'epoca 15: 0.8571

--- FOLD 5/10 ---
Early stopping attivato all'epoca 13!
Fold 5 terminato. Miglior F1 all'epoca 3: 0.8372

--- FOLD 6/10 ---
Early stopping attivato all'epoca 14!
Fold 6 terminato. Miglior F1 all'epoca 5: 0.8235

--- FOLD 7/10 ---
Early stopping attivato all'epoca 15!
Fold 7 terminato. Miglior F1 all'epoca 10